# Model input ERP format preparation 

In this notebook: 
- Necessary inputs
- Read all epochs
- Function to create dataframe with average mismatch response for all participants (needs to be transformed to function)
- Formatting dataframe as suitable model input

## Imports

In [1]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import ipywidgets as widgets
from IPython.display import display

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import sys
sys.path.insert(0, '../eegyolk') # path to helper functions
#import eegyolk
import helper_functions as hf # library useful for eeg and erp data cleaning
import initialization_functions #library to import data
import epod_helper

In [2]:
metadata = pd.read_csv('metadata.csv', sep = ',')

In [3]:
metadata.head()

,eeg_file,ParticipantID,test,sex,age_months,dyslexic_parent,Group_AccToParents,path_eeg,path_epoch,path_eventmarkers,epoch_file
0,105a,105,a,f,17,f,At risk,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,105a_epo.fif
1,107a,107,a,f,16,m,At risk,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,107a_epo.fif
2,106a,106,a,m,19,f,At risk,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,106a_epo.fif
3,109a,109,a,m,21,m,At risk,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,109a_epo.fif
4,110a,110,a,m,17,m,At risk,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,110a_epo.fif


## Read all epochs from files

The function below loads all filtered epochs from the dataframe which contains the metadata and the epochs filepaths + filenames. The epochs are arrays for each stimuli with a time interval of -0.2 to 0.8.

In [4]:
epochs = initialization_functions.read_filtered_data(metadata)

Checking out file: 105a_epo.fif
Checking out file: 107a_epo.fif
Checking out file: 106a_epo.fif
Checking out file: 109a_epo.fif
Checking out file: 110a_epo.fif
Checking out file: 112a_epo.fif
Checking out file: 111a_epo.fif
Checking out file: 114a_epo.fif
Checking out file: 115a_epo.fif
Checking out file: 117a_epo.fif
Checking out file: 116a_epo.fif
Checking out file: 118a_epo.fif
Checking out file: 119a_epo.fif
Checking out file: 123a_epo.fif
Checking out file: 122a_epo.fif
Checking out file: 124a_epo.fif
Checking out file: 127a_epo.fif
Checking out file: 125a_epo.fif
Checking out file: 126a_epo.fif
Checking out file: 130a_epo.fif
Checking out file: 128a_epo.fif
Checking out file: 129a_epo.fif
Checking out file: 131a_epo.fif
Checking out file: 135a_epo.fif
Checking out file: 133a_epo.fif
Checking out file: 137a_epo.fif
Checking out file: 138a_epo.fif
Checking out file: 139a_epo.fif
Checking out file: 141a_epo.fif
Checking out file: 144a_epo.fif
Checking out file: 143a_epo.fif
Checking

In [5]:
len(epochs)

101

## Create pandas dataframe with the average difference between standard and deviant responses

The function below needs `metadata`, the loaded `epochs` and the definition of the standard and deviant events as input. You should define your standard and deviant events as an array. In the function `input_mmr_prep` it's important to know that the assumption is made that the deviant follows after a standard event. Therefore the deviant belonging to the standard is the  standard event number + 1. Make sure your events are numbered like this, else the function won't calculate the mismatch response.  

In [6]:
def input_mmr_prep(metadata, epochs, standard_events, deviant_events): 
    # create dataframe with expected columns 
    df = pd.DataFrame(columns=["eeg_file",  "channel", "mean"]) # "paradigm",

    # loop over all participants
    for i in range(len(metadata['eeg_file'])):
        std_evoked = epochs[i][standard_events].average() 
        dev_evoked = epochs[i][deviant_events].average()

        # calculate the mismatch response between standard and deviant evoked
        evoked_diff = mne.combine_evoked([std_evoked, dev_evoked], weights=[1, -1])
        
        # get a list of all channels
        chnames_list = evoked_diff.info['ch_names']
        
        # compute for every channel the features of the mismatch line
        for channel in chnames_list: 
            chnames = mne.pick_channels(evoked_diff.info['ch_names'], include=[channel])
            roi_dict = dict(left_ROI=chnames) # combine_channels only takes a dictionary as input
            roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
            mmr = roi_evoked.to_data_frame()
            mmr_avg = mmr['left_ROI'].mean()
            mmr_std = mmr['left_ROI'].std()
            mmr_skew = mmr['left_ROI'].skew()
            mmr_var = mmr['left_ROI'].var()
            mmr_kurt = mmr['left_ROI'].kurtosis()
            
            df = df.append({'eeg_file': metadata['eeg_file'][i], 'channel': channel, 'mean' :  mmr_avg, 'std' : mmr_std, 'skew' : mmr_skew, 'kurt' : mmr_kurt, 'var' : mmr_var}, ignore_index=True) # add 'paradigm : paradigm' if we want to separate the paradigms 
    return df

In [7]:
# define the events for standard and deviant
standard_events = ['GiepM_S','GiepS_S','GopM_S','GopS_S']
deviant_events = ['GiepM_D','GiepS_D','GopM_D','GopS_D']


df = input_mmr_prep(metadata, epochs, standard_events, deviant_events)

/tmp/ipykernel_1175280/1378168468.py:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x7f98bc12ee50>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
/tmp/ipykernel_1175280/1378168468.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'eeg_file': metadata['eeg_file'][i], 'channel': channel, 'mean' :  mmr_avg, 'std' : mmr_std, 'skew' : mmr_skew, 'kurt' : mmr_kurt, 'var' : mmr_var}, ignore_index=True) # add 'paradigm : paradigm' if we want to separate the paradigms
/tmp/ipykernel_1175280/1378168468.py:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x7f98bc12ee50>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
/tmp/ipykernel_1175280/1378168468.p

In [8]:
df

,eeg_file,channel,mean,kurt,skew,std,var
0,105a,Fp1,1.231969,-0.335317,0.680321,2.775716,7.704598
1,105a,AF3,1.518115,0.295970,0.835901,1.957988,3.833719
2,105a,F7,4.762359,0.009801,0.904384,4.515845,20.392858
3,105a,F3,2.544221,0.195387,0.932187,2.576173,6.636665
4,105a,FC1,-0.375663,-0.471158,-0.395098,0.610872,0.373165
...,...,...,...,...,...,...,...
3227,221a,F8,-0.084982,-0.048255,-0.138260,1.078537,1.163242
3228,221a,AF4,-0.290859,-0.191029,-0.248536,1.043134,1.088128
3229,221a,Fp2,-0.260251,0.339752,-0.408454,0.877273,0.769608
3230,221a,Fz,-0.836911,-0.738132,-0.023938,0.973992,0.948660


In [9]:
df = df.drop_duplicates(subset=['eeg_file','channel']) # ,'paradigm'

## Transpose dataframe into combination of paradigm and channel per participant

We now want a single row for every participant containing the paradigm and corresponding channels. The code below generates this dataframe. 

In [10]:
# transformation of the dataframe
df = df.pivot(index='eeg_file', columns=['channel']) # 'paradigm',

In [11]:
df.columns = ['_'.join(str(s).strip() for s in col if s) for col in df.columns]

In [12]:
df.reset_index(inplace=True)

## Merge and safe dataframe

We still need to merge some of the metadata into the dataframe, so we have the information of the age, gender and label of the participant. 

In [13]:
df = pd.merge(df, metadata, on='eeg_file')

In [14]:
pd.set_option('display.max_columns', None)

In [15]:
df

,eeg_file,mean_AF3,mean_AF4,mean_C3,mean_C4,mean_CP1,mean_CP2,mean_CP5,mean_CP6,mean_Cz,mean_F3,mean_F4,mean_F7,mean_F8,mean_FC1,mean_FC2,mean_FC5,mean_FC6,mean_Fp1,mean_Fp2,mean_Fz,mean_O1,mean_O2,mean_Oz,mean_P3,mean_P4,mean_P7,mean_P8,mean_PO3,mean_PO4,mean_Pz,mean_T7,mean_T8,kurt_AF3,kurt_AF4,kurt_C3,kurt_C4,kurt_CP1,kurt_CP2,kurt_CP5,kurt_CP6,kurt_Cz,kurt_F3,kurt_F4,kurt_F7,kurt_F8,kurt_FC1,kurt_FC2,kurt_FC5,kurt_FC6,kurt_Fp1,kurt_Fp2,kurt_Fz,kurt_O1,kurt_O2,kurt_Oz,kurt_P3,kurt_P4,kurt_P7,kurt_P8,kurt_PO3,kurt_PO4,kurt_Pz,kurt_T7,kurt_T8,skew_AF3,skew_AF4,skew_C3,skew_C4,skew_CP1,skew_CP2,skew_CP5,skew_CP6,skew_Cz,skew_F3,skew_F4,skew_F7,skew_F8,skew_FC1,skew_FC2,skew_FC5,skew_FC6,skew_Fp1,skew_Fp2,skew_Fz,skew_O1,skew_O2,skew_Oz,skew_P3,skew_P4,skew_P7,skew_P8,skew_PO3,skew_PO4,skew_Pz,skew_T7,skew_T8,std_AF3,std_AF4,std_C3,std_C4,std_CP1,std_CP2,std_CP5,std_CP6,std_Cz,std_F3,std_F4,std_F7,std_F8,std_FC1,std_FC2,std_FC5,std_FC6,std_Fp1,std_Fp2,std_Fz,std_O1,std_O2,std_Oz,std_P3,std_P4,std_P7,std_P8,std_PO3,std_PO4,std_Pz,std_T7,std_T8,var_AF3,var_AF4,var_C3,var_C4,var_CP1,var_CP2,var_CP5,var_CP6,var_Cz,var_F3,var_F4,var_F7,var_F8,var_FC1,var_FC2,var_FC5,var_FC6,var_Fp1,var_Fp2,var_Fz,var_O1,var_O2,var_Oz,var_P3,var_P4,var_P7,var_P8,var_PO3,var_PO4,var_Pz,var_T7,var_T8,ParticipantID,test,sex,age_months,dyslexic_parent,Group_AccToParents,path_eeg,path_epoch,path_eventmarkers,epoch_file
0,101a,-0.307508,-1.117406,-0.265137,-2.490048,-0.344755,-1.952034,0.244291,-2.060369,-0.430365,-0.360602,-0.869062,-1.685437,-1.320273,0.037099,-0.283584,-0.677674,-1.250907,-1.169016,-1.749117,-0.033056,-0.597666,-2.596950,-2.514119,-0.644141,-2.824247,0.013660,-1.826525,-1.024139,-2.431471,-1.670712,-1.152493,-1.818161,-0.662227,0.552862,-0.722270,-0.902288,0.417584,0.108099,0.066907,-0.898025,-0.489093,0.142669,-0.200325,0.040356,-0.728821,-0.704019,-0.353652,0.729763,-0.599536,0.260606,0.290342,-0.457298,-1.108639,-1.227771,-1.250857,0.540389,-1.137817,-0.300789,-1.169184,-1.043322,-1.059768,-0.677592,-1.123509,-0.569833,-0.630391,-0.870358,0.076734,0.417313,-0.563143,0.990278,0.392878,-0.310648,0.246699,-1.027285,-0.422932,-0.656505,-0.178348,0.094718,0.455089,-1.048427,-0.336491,-1.038757,-0.790867,-0.375478,-0.213395,-0.205587,-0.440525,-0.434018,0.012159,0.192717,-0.315029,-0.554302,-0.230594,-0.310189,0.221245,-0.596817,1.361383,1.384671,0.480952,1.595303,0.502096,1.123514,0.749415,1.692374,0.410975,1.129224,1.003108,2.439540,1.303306,0.456107,0.501310,1.020202,1.320292,1.613613,1.632540,1.002481,1.934221,2.332280,2.525475,0.858019,2.064744,0.988334,2.020275,1.680420,1.925573,1.365854,1.116285,2.039052,1.853363,1.917314,0.231314,2.544990,0.252101,1.262284,0.561623,2.864129,0.168900,1.275148,1.006226,5.951354,1.698606,0.208033,0.251312,1.040812,1.743170,2.603748,2.665188,1.004969,3.741212,5.439529,6.378025,0.736197,4.263168,0.976805,4.081511,2.823812,3.707833,1.865558,1.246092,4.157734,101,a,m,20,m,At risk,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,101a_epo.fif
1,103a,-1.565712,-0.481950,0.163128,2.782407,2.052576,3.038309,-2.244892,4.449279,1.505256,-0.468584,-0.409927,-1.775422,-0.524874,2.545403,4.189770,-1.517147,1.984296,-1.604687,1.406518,1.873731,16.137246,6.154369,7.412724,1.583712,2.945922,3.488876,6.081199,1.640105,3.867000,4.153652,-2.908292,2.597282,-0.492150,-0.621200,-0.894298,-1.337962,-1.477607,0.888663,0.078056,-1.551199,-1.419022,0.033330,-0.637463,-0.691826,0.577421,-1.265332,-1.405419,-0.384776,-1.104496,-0.506498,-0.755672,-1.199144,-1.510031,1.763832,-1.301725,-1.216397,-1.221052,-0.492610,-1.202709,-0.142656,-1.012291,-1.404193,-0.394864,-1.269425,-0.253335,-0.333204,0.025692,-0.336970,-0.218370,-0.202691,0.104599,-0.270573,-0.163093,-0.706264,-0.317889,-0.307560,-0.798121,-0.041694,-0.439783,-0.609067,-0.302764,-0.249415,-0.152087,-0.317376,-0.094585,0.784048,-0.272050,-0.149114,0.216997,-0.517542,-0.225026,

Drop some unnecessary columns. 

In [16]:
df = df.drop(['eeg_file',
       'dyslexic_parent', 'path_eeg','path_epoch',
       'epoch_file', 'path_eventmarkers'], axis =1)

In [17]:
df['sex'] = np.where(
    (df['sex']=='m'), 1,0)

df['Group_AccToParents'] = np.where(
    (df['Group_AccToParents']=='At risk'), 1,0)

In [18]:
first = df.pop('Group_AccToParents')
df.insert(0, 'Group_AccToParents', first)

In [19]:
df.to_csv('df_avg_mmr.csv', index=False) # safe dataframe

## PCA analysis on feature reduction 

In [20]:
X = df.drop('Group_AccToParents',1)
y = df['Group_AccToParents']

/tmp/ipykernel_1175280/1933137523.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = df.drop('Group_AccToParents',1)


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [22]:
pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

ValueError: could not convert string to float: 'a'

In [23]:
explained_variance = pca.explained_variance_ratio_

AttributeError: 'PCA' object has no attribute 'explained_variance_ratio_'

In [ ]:
explained_variance

In [ ]:
classifier = RandomForestClassifier(max_depth=2, random_state=0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))

In [ ]:
X.shape